In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker
import seaborn as sns
%load_ext autoreload
%autoreload 2

from settings import *
from glove import *
import re
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold
from textblob import TextBlob

In [ ]:
# df = pd.read_csv(cache_dir+'similarity_model/test_pairs_v3.tsv', sep='\t')

# df = pd.DataFrame(df.groupby(['article'])['related'].apply(list).reset_index()['related'])

# df['num_of_links'] = df['related'].apply(len)

# def count_True(l):
#     total = len(l)
#     trues = len([1 for e in l if e])
#     return trues/total


# df['perc'] = df['related'].apply(count_True)

# df.to_csv('/Users/smeros/Desktop/test.tsv', sep='\t', index=None)

# df[df['num_of_links']==4].to_csv('/Users/smeros/Desktop/test_4.tsv', sep='\t', index=None)

In [ ]:
# snopes = pd.read_csv('/home/psmeros/Dropbox/scilens/snopes-u.csv')
# articles = pd.read_csv(cache_dir+'article_details_v5.tsv', sep='\t')

# def get_snopes_articles(title, snopes_list, snopes_list_vec):
#     for sno, vec in zip(snopes_list, snopes_list_vec):
#         title_vec = sent2vec(title)
#         sim = cos_sim(vec, title_vec)
#         if sim > 0.8:
#             print('title:', title)
#             print('snopes:', sno)
#             print(sim)
#     return


# articles.apply(lambda x: get_snopes_articles(x['title'], snopes['claim'].tolist(), snopes['claim'].apply(sent2vec).tolist()), axis=1)

In [ ]:
df = pd.read_csv('/home/psmeros/Dropbox/scilens/corpus/annotated_replies.csv')

In [ ]:
df['sim'] = df.apply(lambda x: cos_sim(sent2vec(x['full_text']), sent2vec(x['reply'])),axis=1)

In [ ]:
df['word_count'] = df['reply'].apply(lambda x: len((re.sub(' +',' ',re.sub(r'[^a-zA-Z0-9 ]', '', x))).strip().split(' ')))

In [ ]:
df['negation'] = df['reply'].apply(lambda x: any(n in x for n in [' no ', ' not ', 'n\'t ']))

In [ ]:
positive_words = open(conf['aux_dir'] + 'small_files/opinion/positive-words.txt', encoding='utf-8', errors='ignore').read().splitlines()
negative_words = open(conf['aux_dir'] + 'small_files/opinion/negative-words.txt', encoding='utf-8', errors='ignore').read().splitlines()
df['positive'] = df['reply'].apply(lambda x: sum(n in x for n in positive_words))
df['negative'] = df['reply'].apply(lambda x: sum(n in x for n in negative_words))

In [ ]:
df['length'] = df['reply'].apply(len)

In [ ]:
df['has_url'] = df['reply'].apply(lambda x: bool(re.search('http(s)?://', x)))

In [ ]:
df['quest_mark'] = df['reply'].apply(lambda x: x.count('?'))

In [ ]:
df['excl_mark'] = df['reply'].apply(lambda x: x.count('!'))

In [ ]:
df['reply_polarity'] =  df['reply'].apply(lambda x: TextBlob(x).sentiment.polarity)
df['reply_subjectivity'] =  df['reply'].apply(lambda x: TextBlob(x).sentiment.subjectivity)
df['full_text_polarity'] =  df['full_text'].apply(lambda x: TextBlob(x).sentiment.polarity)
df['full_text_subjectivity'] =  df['full_text'].apply(lambda x: TextBlob(x).sentiment.subjectivity)

In [ ]:
df['stance'] = df['stance'].apply(lambda x: 'con-quest' if x in ['con', 'quest'] else x).apply(lambda x: 'nr-com' if x in ['nr', 'com']  else x)

In [ ]:
X = np.array(df.drop(['full_text', 'reply', 'stance'], axis=1).values, dtype=np.float32)
y = df['stance'].values

In [ ]:
fold = 5
kf = KFold(n_splits=fold, shuffle=True)
score = 0.0
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    #X_train = (X_train - X_train.mean(axis=0)) / (X_train.std(axis=0)+1e-9)
    #X_test = (X_test - X_test.mean(axis=0)) / (X_test.std(axis=0)+1e-9)
    n_est = 50
    m_dep = 10
    #from sklearn.svm import LinearSVC
    #classifier = LinearSVC(C=0.1, max_iter=10000)
    classifier = RandomForestClassifier(n_estimators=n_est, max_depth=m_dep, n_jobs=-1)
    classifier.fit(X_train, y_train)
    score += classifier.score(X_test, y_test)

print('Score:', score/fold)
